In [1]:
!pip install ortools==9.9.3963

  Obtaining dependency information for ortools==9.9.3963 from https://files.pythonhosted.org/packages/1c/51/47ed569a9ad312863090d0350a8a4ddb0291f1d5e777f594029678c6fc57/ortools-9.9.3963-cp311-cp311-macosx_10_15_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=2.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf>=4.25.3 from https://files.pythonhosted.org/packages/1e/40/2eb2bf643d4b060b1602a25748b48d75431f4951be2470f8ae136952b3d3/protobuf-5.26.1-cp37-abi3-macosx_10_9_universal2.whl.metadata
  Obtaining dependency information for immutabledict>=3.0.0 from https://files.pythonhosted.org/packages/e2/13/3cf4ac5b3403f3456e645c4533883ef67b1bb0c72e56b79c707715f57a74/immutabledict-4.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 5.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
def parse_vrp_instance(file_path, num_vehicles=1):
    """Parses the VRP instance from a text file."""
    with open(file_path, 'r') as file:
        lines = file.readlines()

    start_index = 0
    for i, line in enumerate(lines):
        if 'CUST NO.' in line:
            start_index = i + 1  # Data starts after headers
            break

    locations = []
    demands = []
    time_windows = []
    service_times = []

    for line in lines[start_index:]:
        if line.strip():
            parts = line.split()
            cust_no, x, y, demand, ready_time, due_date, service_time = map(int, parts)
            locations.append((x, y))
            demands.append(demand)
            time_windows.append((ready_time, due_date))
            service_times.append(service_time)

    return {
        'locations': locations,
        'demands': demands,
        'time_windows': time_windows,
        'service_times': service_times,
        'vehicle_capacities': [200] * num_vehicles,
        'num_vehicles': num_vehicles,
        'depot': 0
    }

file_path = 'instance_1.txt'
data_model = parse_vrp_instance(file_path)
print(data_model)


{'locations': [(40, 50), (45, 68), (45, 70), (42, 66), (42, 68), (42, 65), (40, 69), (40, 66), (38, 68), (38, 70), (35, 66), (35, 69), (25, 85), (22, 75), (22, 85), (20, 80), (20, 85), (18, 75), (15, 75), (15, 80), (30, 50), (30, 52), (28, 52), (28, 55), (25, 50), (25, 52)], 'demands': [0, 10, 30, 10, 10, 10, 20, 20, 20, 10, 10, 10, 20, 30, 10, 40, 40, 20, 20, 10, 10, 20, 20, 10, 10, 40], 'time_windows': [(0, 1236), (912, 967), (825, 870), (65, 146), (727, 782), (15, 67), (621, 702), (170, 225), (255, 324), (534, 605), (357, 410), (448, 505), (652, 721), (30, 92), (567, 620), (384, 429), (475, 528), (99, 148), (179, 254), (278, 345), (10, 73), (914, 965), (812, 883), (732, 777), (65, 144), (169, 224)], 'service_times': [0, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90], 'vehicle_capacities': [200], 'num_vehicles': 1, 'depot': 0}


In [3]:

def print_solution(data, manager, routing, solution):
    print('Objective: {}'.format(solution.ObjectiveValue()))
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index), route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total Distance of all routes: {}m'.format(total_distance))
    print('Total Load of all routes: {}'.format(total_load))

In [7]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def compute_euclidean_distance_matrix(locations):
    distances = {}
    for from_counter, from_node in enumerate(locations):
        distances[from_counter] = {}
        for to_counter, to_node in enumerate(locations):
            distances[from_counter][to_counter] = (abs(from_node[0] - to_node[0]) + abs(from_node[1] - to_node[1]))
    return distances

def main():
    data = parse_vrp_instance(file_path, num_vehicles=10)

    manager = pywrapcp.RoutingIndexManager(len(data['locations']), data['num_vehicles'], data['depot'])

    routing = pywrapcp.RoutingModel(manager)

    distance_matrix = compute_euclidean_distance_matrix(data['locations'])

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        0,  # allow waiting time
        999,  # maximum time per vehicle # TODO adjust?
        False,  # Don't force start cumul to zero.
        time)

    time_dimension = routing.GetDimensionOrDie(time)
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][depot_idx][0],
                                                data['time_windows'][depot_idx][1])

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 10

    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("no solution found xd")

main()


Objective: 878
Route for vehicle 0:
 0 Load(0) ->  8 Load(20) ->  19 Load(30) ->  10 Load(40) ->  15 Load(80) ->  0 Load(80)
Distance of the route: 168m
Load of the route: 80

Route for vehicle 1:
 0 Load(0) ->  21 Load(20) ->  1 Load(30) ->  0 Load(30)
Distance of the route: 66m
Load of the route: 30

Route for vehicle 2:
 0 Load(0) ->  24 Load(10) ->  18 Load(30) ->  7 Load(50) ->  0 Load(50)
Distance of the route: 100m
Load of the route: 50

Route for vehicle 3:
 0 Load(0) ->  11 Load(10) ->  16 Load(50) ->  9 Load(60) ->  14 Load(70) ->  6 Load(90) ->  12 Load(110) ->  0 Load(110)
Distance of the route: 234m
Load of the route: 110

Route for vehicle 4:
 0 Load(0) ->  20 Load(10) ->  5 Load(20) ->  13 Load(50) ->  3 Load(60) ->  17 Load(80) ->  25 Load(120) ->  0 Load(120)
Distance of the route: 176m
Load of the route: 120

Route for vehicle 5:
 0 Load(0) ->  23 Load(10) ->  4 Load(20) ->  22 Load(40) ->  2 Load(70) ->  0 Load(70)
Distance of the route: 134m
Load of the route: 70

R